# H-Infinity Robust Controller for Vertical Stability

Demonstrates the Doyle-Glover-Khargonekar H-infinity synthesis implemented in
`scpn_control.control.h_infinity_controller`. The controller:

- Solves two continuous algebraic Riccati equations (AREs) for gamma feasibility
- Derives discrete-time gains via DARE on ZOH-discretised plant at each sampling rate
- Provides output saturation with anti-windup back-calculation
- Guarantees closed-loop stability for any sampling dt (not just the design point)

**Dependencies:** numpy, scipy, matplotlib (no GPU, no optional deps)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scpn_control.control.h_infinity_controller import (
    HInfinityController,
    get_radial_robust_controller,
)

def zoh_discretize(A, B, dt):
    """Exact ZOH discretisation via matrix exponential."""
    n, m = A.shape[0], B.shape[1]
    M = np.zeros((n + m, n + m))
    M[:n, :n] = A * dt
    M[:n, n:] = B * dt
    eM = expm(M)
    return eM[:n, :n], eM[:n, n:]

ctrl = get_radial_robust_controller(gamma_growth=100.0, damping=10.0)
print(f"gamma           = {ctrl.gamma:.1f}")
print(f"is_stable       = {ctrl.is_stable}")
print(f"robust_feasible = {ctrl.robust_feasible}")
print(f"gain_margin     = {ctrl.gain_margin_db:.2f} dB")
res_x, res_y = ctrl.riccati_residual_norms()
print(f"Riccati res X   = {res_x:.2e}")
print(f"Riccati res Y   = {res_y:.2e}")
print(f"feasibility margin = {ctrl.robust_feasibility_margin():.2e}")

## Step Response — Vertical Instability Stabilisation

The open-loop plant has an eigenvalue at +100 (exponential growth).
The H-infinity controller drives the initial displacement to zero.

In [ ]:
dt = 0.05
n_steps = 600
A, B2, C2 = ctrl.A, ctrl.B2, ctrl.C2
Ad, Bd = zoh_discretize(A, B2, dt)

ctrl.reset()
x = np.array([0.1, 0.0])
errors = []
controls = []
for _ in range(n_steps):
    y = (C2 @ x).item()
    u = ctrl.step(y, dt)
    x = Ad @ x + Bd.ravel() * u
    errors.append(y)
    controls.append(u)

t = np.arange(n_steps) * dt
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
ax1.plot(t, errors, "b-", lw=1.5)
ax1.set_ylabel("Position error [m]")
ax1.set_title("H-inf Step Response (dt=50ms, plant eigenvalue +100)")
ax1.axhline(0, color="gray", ls="--", lw=0.5)
ax1.grid(True, alpha=0.3)

ax2.plot(t, controls, "r-", lw=1)
ax2.set_ylabel("Control u [A]")
ax2.set_xlabel("Time [s]")
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print(f"Converged by step {next(i for i,e in enumerate(errors) if abs(e) < 0.01*abs(errors[0]))}  (error < 1% of initial)")

## Disturbance Rejection — 10 MA Current Kick at t=5s

In [ ]:
ctrl.reset()
B1 = ctrl.B1
_, Bd_w = zoh_discretize(A, B1, dt)

x = np.array([0.0, 0.0])
errors_d = []
for i in range(n_steps):
    t_now = i * dt
    y = (C2 @ x).item()
    u = ctrl.step(y, dt)
    w = 10.0 if abs(t_now - 5.0) < dt else 0.0
    x = Ad @ x + Bd.ravel() * u + Bd_w.ravel() * w
    errors_d.append(y)

t = np.arange(n_steps) * dt
plt.figure(figsize=(10, 4))
plt.plot(t, errors_d, "b-", lw=1.5)
plt.axvline(5.0, color="red", ls="--", lw=1, label="10 MA kick")
plt.xlabel("Time [s]")
plt.ylabel("Position [m]")
plt.title("Disturbance Rejection")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
print(f"Final error after kick: {abs(errors_d[-1]):.2e}")

## Multi-dt Stability — DARE Guarantees

The DARE-based discretisation guarantees stability at any sampling rate.
Forward Euler would diverge at dt=0.07 (step amplification > 780x).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (dt_val, n_sim, label) in zip(axes, [
    (0.001, 5000, "dt=1ms (1 kHz)"),
    (0.05, 600, "dt=50ms (20 Hz)"),
    (0.07, 500, "dt=70ms (14 Hz)"),
]):
    ctrl.reset()
    Ad_i, Bd_i = zoh_discretize(A, B2, dt_val)
    x = np.array([0.1, 0.0])
    errs = []
    diverged = False
    for _ in range(n_sim):
        y = (C2 @ x).item()
        if not np.isfinite(y):
            diverged = True
            break
        u = ctrl.step(y, dt_val)
        x = Ad_i @ x + Bd_i.ravel() * u
        errs.append(abs(y))

    t = np.arange(len(errs)) * dt_val
    ax.semilogy(t, errs, lw=1)
    ax.set_title(label)
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("|error|")
    ax.grid(True, alpha=0.3)
    ax.set_ylim(bottom=1e-16)
    if diverged:
        ax.annotate("diverged (overflow)", xy=(0.95, 0.95), xycoords="axes fraction",
                    ha="right", va="top", fontsize=9, color="red")
    else:
        final = errs[-1]
        ax.annotate(f"final={final:.1e}", xy=(0.95, 0.95), xycoords="axes fraction",
                    ha="right", va="top", fontsize=9)

plt.suptitle("DARE-based H-inf: Multi-Rate Stability Test", fontsize=13)
plt.tight_layout()
plt.show()

## Custom Plant — User-Defined State-Space

Build an H-infinity controller for an arbitrary SISO plant.

In [ ]:
A_custom = np.array([
    [0.0,   1.0,  0.0],
    [0.0,   0.0,  1.0],
    [-6.0, -11.0, -6.0],
])
B1_custom = np.array([[0.0], [0.0], [1.0]])
B2_custom = np.array([[0.0], [1.0], [0.0]])
C1_custom = np.array([[1.0, 0.0, 0.0]])
C2_custom = np.array([[1.0, 0.0, 0.0]])

ctrl3 = HInfinityController(
    A_custom, B1_custom, B2_custom, C1_custom, C2_custom,
)
print(f"3-state plant: gamma={ctrl3.gamma:.1f}, stable={ctrl3.is_stable}, "
      f"GM={ctrl3.gain_margin_db:.1f} dB")

dt3 = 0.01
Ad3, Bd3 = zoh_discretize(A_custom, B2_custom, dt3)
ctrl3.reset()
x = np.array([1.0, 0.0, 0.0])
errs3 = []
for _ in range(1000):
    y = (C2_custom @ x).item()
    u = ctrl3.step(y, dt3)
    x = Ad3 @ x + Bd3.ravel() * u
    errs3.append(y)

t3 = np.arange(1000) * dt3
plt.figure(figsize=(10, 3))
plt.plot(t3, errs3, lw=1.5)
plt.xlabel("Time [s]"); plt.ylabel("Output")
plt.title("Custom 3-State Plant — H-inf Regulation")
plt.grid(True, alpha=0.3)
plt.show()
print(f"Final |error| = {abs(errs3[-1]):.2e}")

---

**Summary:** The H-infinity controller in `scpn_control` provides:

1. Automatic gamma bisection with Riccati feasibility verification
2. DARE-based discretisation — stable at any sampling rate
3. Output saturation with anti-windup back-calculation
4. Works for arbitrary SISO state-space plants (2-state, 3-state, ...)

See `scpn_control.control.h_infinity_controller` for the full API.